# English and Spanish used cars sales description using SK<br/>Text generation scenario

Learning objectives:

- What does SK provide for me?
  - A configurable Kernel with pluggable architecture
  - The models to work with the OpenAI endpoints
  - A resilient HttpClient that can handle disconnections and throttlihg
  - SK functions that can be pipe to achieve complex orchestrations such a text generation and transaltion
  - Ability to handle prompts and completions

## Load the required .NET packages and supporting classes

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.3.0"
#r "nuget: Microsoft.SemanticKernel.Core, 1.3.0"

#r "nuget: dotenv.net, 3.1.2"

using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using dotenv.net;

#!import Models/Models.cs

// Json Options
readonly JsonSerializerOptions s_jsonOptions = new() { WriteIndented = true };

## Load the OpenAI endpoint and API key from environment variables or an .env file

In [ ]:
DotEnv.Load();
var deploymentName = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
Console.WriteLine($"Using deployment: {deploymentName} at: {endpoint} with key {apiKey.Substring(0, 5)}...");

## Mock a database call to get a list of car

In [ ]:
var mockCarData = new List<Car>{
    new Car("123","Ford", "Explorer", 2015, "Silver", "V6", "Platinum", 10000, 16500),
    new Car("456","Ford", "Mustang", 2018, "Blue", "V8", "Sports", 10000, 250000),
    new Car("789","Ford", "Escape", 2020, "Red", "V6","Special", 3000, 15000)
  };
  
List<Car> MockDBCall() {  
  return mockCarData;
}

## Function to get a short car description from the car properties

In [ ]:
// Reminder: I created this function, to make more impact, but the car record has a ToString defition to return the same information
string GetCarDescription(Car car) {
  return $"{car.Year} {car.Make} {car.Model} {car.Color} {car.Motor} {car.Package} with {car.milage} miles for ${car.price}";
}

## Create an instance of a kernel

- Here notice how elegantly the kernel is configured

In [ ]:
var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(deploymentName, endpoint, apiKey)
    .Build();

## Create the sales definition and translation in-line functions and add them to the kernel

In [ ]:
// Define a function to get a used car sales description
const string fnSalesDefinition = @"Get a used car sales description for the following: 
{{$input}}.";

var salesDescriptionFunc = kernel.CreateFunctionFromPrompt(fnSalesDefinition, 
    new OpenAIPromptExecutionSettings() { MaxTokens = 200, Temperature = 0.5, TopP = 1 });

// Define a function to translate English to Spanish
const string fnESTranslatorDefinition = @"Translate the following English text to Spanish: 
{{$input}}.";

var esTranslatorFunc = kernel.CreateFunctionFromPrompt(fnESTranslatorDefinition, 
    new OpenAIPromptExecutionSettings() { MaxTokens = 200, Temperature = 0.5, TopP = 1 });

## Using the kernel and sk functions, get a car sales description and translate it into Spanish

In [ ]:
// Mock making a call to a database to get a list of cars
var cars = MockDBCall();
var carDesc = GetCarDescription(cars[0]);
cars[0].ToString()

## Get a sales description and a translation for all the cars in the mock database

In [ ]:
// Keep a list of processed cars
var processedCars = new List<CarTranslation>();

foreach(var car in cars) {

  // Get a short description given the car properties
  var desc = GetCarDescription(car);
  Console.WriteLine($"Processing:\n{desc}");
  
  // Use the SK sales description function to get a sales for a car
  var arguments = new KernelArguments()
        {
            ["input"] = desc
        };
  var result = await kernel.InvokeAsync(salesDescriptionFunc, arguments);
  Console.WriteLine($"Sales Description:\n{result}");

  // Get the usage for the call
  //Console.WriteLine(JsonSerializer.Serialize(result.FunctionResults.LastOrDefault()?.GetOpenAIChatResult()?.Usage, s_jsonOptions));  
  
  // Use the SK translation function to translate the sales description into Spanish
  arguments.Clear();
  arguments["input"] = result.ToString();
  var esTranslation = await kernel.InvokeAsync(esTranslatorFunc,arguments);
  Console.WriteLine($"Translation:\n{esTranslation}\n");
  
  // Add processed the EN and ES sales description
  processedCars.Add(new CarTranslation(car.VIN, result.ToString(), esTranslation.ToString()));
}

// Show the processed cars
processedCars